In [ ]:
import os

import torch
from torch.cuda.amp import GradScaler, autocast
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from ds_handlers.dataset_ball_only_triplet_frames import DatasetBallOnlyTripletFrames
from models.tracknet.tracknet import TrackNet


In [2]:
# trying changes

In [ ]:

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# OPTIONAL -> DOWNLOADING DATASET FROM GDRIVE
# !pip -q install gdown
# !gdown --id 1DQ3ZbvokTsgOq6x-ay6O8U2W4a8e3LFw -O Dataset.zip
# !unzip Dataset.zip

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = TrackNet(in_channels=9, num_bins=1).to(device)

In [ ]:
DATASET_ROOT_PATH = 'NEED_TO_FILL_THIS!!!!!!!!'


train_ds = DatasetBallOnlyTripletFrames(DATASET_ROOT_PATH, out_size=(352, 640))
val_split = int(len(train_ds) * 0.1)
train_subset, val_subset = torch.utils.data.random_split(
    train_ds, [len(train_ds) - val_split, val_split], generator=torch.Generator().manual_seed(42)
)

train_loader = DataLoader(train_subset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_subset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True, persistent_workers=True)

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
scaler = GradScaler(enabled=torch.cuda.is_available())